# XGBoost with carbon isotopes  TEST NOTEBOOK

In [1]:
# Imports
import joblib
import pandas as pd

from numpy import loadtxt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## Read the CSV

In [2]:
# Read csv file
carbon = pd.read_csv('presolargrains_C_XBG.csv')
carbon.head()

,Type,12C/13C
0,4,9455.10
1,4,6793.00
2,4,6227.00
3,4,4250.00
4,4,3993.46


## Train Test Split

In [3]:
# Split data into X and y
X = carbon.loc[::1,'12C/13C'].values.reshape(-1, 1)
y = carbon['Type']
print(X)

[[9.4551e+03]
 [6.7930e+03]
 [6.2270e+03]
 ...
 [1.5700e+00]
 [1.4200e+00]
 [1.3000e+00]]


In [4]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(11716, 1)

## Fit model

In [5]:
# fit model
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=None, subsample=1, tree_method=None,
       validate_parameters=False, verbosity=None)

In [6]:
# Print model
print(model)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=None, subsample=1, tree_method=None,
       validate_parameters=False, verbosity=None)


In [7]:
# Make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [8]:
# Evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 93.96%


In [9]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_test, y_pred))
rms

0.8549425590727578

## Summary:

In this notebook I wanted to check out an error calculation for the accuracy. However, after working through this, I realized it doesn't make sense to use root mean square error, for example. It converted the y Type values from letters to numbers and then checks the distance between those, but that doesn't make sense in this case. A 1 is further from an 8 than it is from a 2, but that doesn't matter to me. What matters is whether it classified it as "correct" or "incorrect" so I'm going to try to find a categorical error metric.